# LAQN Updated Active sites/species File Function:
- laqn_remove notebook got slower so i will move the update function here.
- Start with paths and modules to import.

In [12]:
import pandas as pd
from pathlib import Path
import re

# paths beloww
base_dir = Path("/Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels")
# the metadata file for nan @value 
nanValue_path = base_dir / "data" / "laqn" / "missing" / "logs_nan_value.csv"

# Month abbreviation list for reference/use in functions
month_list = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# analyse_affected_sites_2023 path below.
check_nonActive_path = base_dir / "data" / "laqn" / "missing" / "notActive_siteSpecies.csv"
output_notActive_siteSpecies_2023 = base_dir / "data" / "laqn" / "missing" / "notActive_siteSpecies_2023.csv"

# checks the removed site-species combinations against active list and removes them
existing_nonactive_path = base_dir / "data" / "laqn" / "missing" / "notActive_site_species.csv"

# analyse_affected_sites_2024 path below.
output_notActive_siteSpecies_2024 = base_dir / "data" / "laqn" / "missing" / "notActive_siteSpecies_2024.csv"

# analyse_affected_sites_2025 path below.
output_notActive_siteSpecies_2025 = base_dir / "data" / "laqn" / "missing" / "notActive_siteSpecies_2025.csv"


## 1) analyse_affected_sites_2023 function to identify site/species with all months missing in 2023
    what does function:

- scans the missing value log for site/species combinations that have 100% missing values for all 12 months in 2023
- if an existing non-active site/species list is provided, only new combinations not already in that list are included
- prints the results and saves them as a CSV file for further review

    paths:

- nanValue_path: base_dir / data / laqn / missing / logs_nan_value.csv
- output_notActive_siteSpecies_2023: base_dir / data / laqn / missing / notActive_siteSpecies_2023.csv
- existing_nonactive_path (optional): base_dir / data / laqn / missing / notActive_site_species.csv
- new csv file save as: notActive_siteSpecies_2023.csv

In [17]:
def analyse_affected_sites_year(
    nanValue_path,  # value_100filtered_missing.csv
    output_notActive_site_species,  # recommended output path
    check_nonActive_path=None,  # notActive_site_species.csv
    year=2025
):
    """
    1. Checks value_100filtered_missing.csv for site/species combos with 100% missing values for all months in the given year.
    2. Compares to notActive_site_species.csv and finds new combos not already listed.
    3. Prints and saves these new combos to output_notActive_site_species.
    """

    def extract_month_number(month_str):
        month_map = {abbr: i+1 for i, abbr in enumerate(month_list)}
        m = re.match(r"\d{4}_(\w{3})", str(month_str).lower())
        if m:
            return month_map.get(m.group(1), None)
        return None

    # Load and filter for year, 100% missing
    df = pd.read_csv(nanValue_path, encoding='utf-8')
    df['month_number'] = df['month'].apply(extract_month_number)
    df = df[df['month_number'].notna()].copy()
    df_year = df[df['year'] == year]
    summary_year = df_year.groupby(['siteCode', 'SpeciesCode'])['month_number'].nunique().reset_index()
    affected_year = summary_year[summary_year['month_number'] == 12].copy()

    print(f"\nTotal site/species combos with 100% missing in {year}: {len(affected_year)}")
    print(affected_year)

    #  Compare to notActive_site_species.csv 
    if check_nonActive_path is not None:
        try:
            existing = pd.read_csv(check_nonActive_path, encoding='utf-8')
            existing_set = set(zip(existing['siteCode'], existing['SpeciesCode']))
            affected_year_set = set(zip(affected_year['siteCode'], affected_year['SpeciesCode']))
            new_combos = affected_year_set - existing_set
            new_affected_year = affected_year[
                affected_year.apply(lambda row: (row['siteCode'], row['SpeciesCode']) in new_combos, axis=1)
            ]
            print(f"\nNew site/species combos NOT in notActive_site_species.csv: {len(new_affected_year)}")
            print(new_affected_year)
            # Save to output
            new_affected_year.to_csv(output_notActive_site_species, index=False, encoding='utf-8')
            print(f"\nSaved new combos to: {output_notActive_site_species}")
            return new_affected_year
        except Exception as e:
            print(f"Warning: Could not filter by notActive_site_species.csv: {e}")
    else:
        print(f"\nNo notActive_site_species.csv provided for comparison.")
    return affected_year


In [18]:
# # Run the function for 2023
# affected_2023 = analyse_affected_sites_year(
#     nanValue_path,
#     output_notActive_siteSpecies_2023,
#     check_nonActive_path=existing_nonactive_path,
#     year=2023
# )

# # Run the function for 2024
# affected_2024 = analyse_affected_sites_year(
#     nanValue_path,
#     output_notActive_siteSpecies_2024,
#     check_nonActive_path=existing_nonactive_path,
#     year=2024
# )

# Run the function for 2025
affected_2025 = analyse_affected_sites_year(
    nanValue_path,
    output_notActive_siteSpecies_2025,
    check_nonActive_path=existing_nonactive_path,
    year=2025
)



Total site/species combos with 100% missing in 2025: 0
Empty DataFrame
Columns: [siteCode, SpeciesCode, month_number]
Index: []

New site/species combos NOT in notActive_site_species.csv: 0
Empty DataFrame
Columns: []
Index: []

Saved new combos to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/missing/notActive_siteSpecies_2025.csv
